In [1]:
import pandas as pd
import spot
import os
import re
from ltlcross_runner import LtlcrossRunner, ResultsAnalyzer

In [2]:
test = False

In [3]:
tools = {
    'ltl2tgba_det': 'ltl2tgba -D -G %f > %O ',
    'ltl3tela_det': '/home/xschwar3/ltl3tela-master/ltl3tela  -D1 -f %f > %O',
    'ltl2tgba': 'ltl2tgba %f > %O ',
    'ltl3tela': '/home/xschwar3/ltl3tela-master/ltl3tela -f %f > %O',
    'delag': '/home/xschwar3/owl/bin/owl ltl2dela -f %f > %O',
    'rabinizer4': '/home/xschwar3/owl/bin/owl ltl2dgra -f %f > %O',
}
tools_order = ['delag', 'rabinizer4', 'ltl2tgba', 'ltl3tela', 'ltl3tela_det']
telatko = '/home/xschwar3/telatko/telatko.py'

In [5]:
patterns_ltl = 'patterns.ltl'
patterns_aut = 'patterns.csv'

In [6]:
if test:
    patterns_ltl = 'test.ltl'

In [7]:
rerun_ltl = True
rerun_aut = True

In [8]:
lcr = LtlcrossRunner(tools, formulas = [patterns_ltl], res_filename = patterns_aut)
if rerun_ltl:
    lcr.run_ltlcross(automata = True, timeout = '60')

In [9]:
ra = ResultsAnalyzer(patterns_aut)

In [10]:
ra.parse_results()

In [11]:
ltlf_translated = {}

for tool in tools_order:
    fs = ra.values.filter(items = [('acc', tool)]).dropna()
    
    ltlf_translated[tool] = len(fs[fs[('acc', tool)] > 1])
    
    fd = open('patterns-' + tool + '.hoa', 'w')
    for (ix, _), _ in fs.iterrows():
        fd.write(ra.aut_for_id(ix, tool).to_str() + '\n')
    fd.close()

In [12]:
os.environ['SPOT_TMPDIR'] = '/home/xschwar3/telatko/concur_response_experiments/pattern-aut/autcross'
os.environ['SPOT_TMPKEEP'] = '1'

def run_autcross(tool):
    !autcross --no-checks --csv=patterns-telatko-{tool}.csv -F patterns-{tool}.hoa 'autfilt %H > %O'   '{telatko} -F %H -T 30 -L 3 -I -G -O %O' 'autfilt %H > %O --simplify-acceptance' 'autfilt %H > %O --small' > autcross/{tool}.log 2>&1

In [4]:
metrics = ['Original', 'Multilevel', 'Autfilt -- simpl-acc', 'Autfilt small']

In [14]:
if rerun_aut:
    run_autcross('delag')

In [15]:
if rerun_aut:
    run_autcross('rabinizer4')

In [16]:
if rerun_aut:
    run_autcross('ltl2tgba')

In [17]:
if rerun_aut:
    run_autcross('ltl3tela')

In [18]:
if rerun_aut:
    run_autcross('ltl3tela_det')

In [ ]:
if rerun_aut:
    run_autcross('ltl2tgba_det')

In [ ]:
csv

In [21]:
data = {}

for tool in tools_order:
    print(tool)
    csv = pd.read_csv('patterns-telatko-' + tool + '.csv')
    
    header = {
        metrics[0]: [],
        metrics[1] : [],
        metrics[2] : [],
        metrics[3] : [],
    }
    i = 0
    for _,r in csv.iterrows():
        
        header[metrics[i]].append(r['output.acc_sets'])
        i = (i + 1) % 4
    data_dd = pd.DataFrame(data = header)
    data_de = data_dd.dropna()
    
    data_df = data_de[(data_de['Original'] > 1)]
    data[tool] = data_df
    print(data_df)

    
    
    
    

delag
     Original  Multilevel  Autfilt -- simpl-acc  Autfilt small
9           2           2                     2              2
10          3           3                     3              3
11          4           4                     4              4
36          3           0                     3              1
44          2           1                     1              1
..        ...         ...                   ...            ...
318         6           6                     6              6
319         8           8                     8              8
320         3           2                     2              2
321         2           1                     1              1
322         3           2                     2              2

[129 rows x 4 columns]
rabinizer4
     Original  Multilevel  Autfilt -- simpl-acc  Autfilt small
0           2           1                     1              1
1           2           1                     1              1
2           2 

In [16]:
data

{}

In [22]:
sumdata = {
    'tool': [],
    'autcount': [],
}
for m in metrics:
    sumdata[m] = []
    sumdata[m + "%"] = []

for tool in tools_order:
    
    data_df = data[tool]
    
    
    sumdata['tool'].append(tool)
    sd = data_df.sum()
    sumdata['autcount'].append(len(data[tool]))
    
    for m in metrics:
        sumdata[m].append(int(sd[m]))
        sumdata[m + "%"].append(str(round(100 - (int(sd[m]) *100/int(sd['Original'])), 1)) + "%")
        
r = pd.DataFrame(data = sumdata)
r

,tool,autcount,Original,Original%,Multilevel,Multilevel%,Autfilt -- simpl-acc,Autfilt -- simpl-acc%,Autfilt small,Autfilt small%
0,delag,129,523,0.0%,379,27.5%,404,22.8%,398,23.9%
1,rabinizer4,234,882,0.0%,541,38.7%,620,29.7%,621,29.6%
2,ltl2tgba,70,198,0.0%,189,4.5%,198,0.0%,198,0.0%
3,ltl3tela,91,348,0.0%,323,7.2%,348,0.0%,348,0.0%
4,ltl3tela_det,81,272,0.0%,272,0.0%,272,0.0%,272,0.0%


In [13]:
data

{}

In [8]:
data = {}

llre = re.compile('^Running \[A([1234])\].*\'(.*?)\'$')
hnre = re.compile('^name: "(.*)"$')
nxre = re.compile('_([TU])\\s*$')

for tool in tools_order:
    csv = pd.read_csv('patterns-telatko-' + tool + '.csv')
    log = 'autcross/' + tool + '.log'
    
    ht = {
#        'aut_id': [],
        'L0.acc': [],
        'L1.acc': [],
        'L1.time': [],
        'L1.qbfr': [],
        'L2.acc': [],
        'L2.time': [],
        'L2.qbfr': [],
        'L3.acc': [],
        'L3.time': [],
        'L3.qbfr': [],
        'L4.acc': [],
        'L4.time': [],
        'L4.qbfr': [],
    }

    #for (ix,_),_ in fs.iterrows():
    #    ht['aut_id'].append(ix)

    i = 0
    for _,r in csv.iterrows():
        ht['L' + str(i) + '.acc'].append(r['output.acc_sets'])
        if i > 0:
            ht['L' + str(i) + '.time'].append(r['time'])

        i = (i + 1) % 5
    
    logfd = open(log, 'r')
    for ll in logfd.readlines():
        lmatch = llre.match(ll)
        if lmatch:
            level = lmatch.group(1)
            hoa = lmatch.group(2)

            hoafd = open(hoa, 'r')
            aut_seen = False
            name_seen = False
            for hl in hoafd.readlines():
                if len(hl) > 3:
                    aut_seen = True
                hmatch = hnre.match(hl)
                if hmatch:
                    name_seen = True
                    xmatch = nxre.search(hmatch.group(1))
                    if xmatch:
                        ht['L' + level + '.qbfr'].append(xmatch.group(1))
                    else:
                        ht['L' + level + '.qbfr'].append('U')
                    break

            hoafd.close()

            if not aut_seen:
                ht['L' + level + '.qbfr'].append('X')
            elif not name_seen:
                ht['L' + level + '.qbfr'].append('U')

    logfd.close()

    #for k in ht:
    #    print(tool, k, len(ht[k]))
    data_dd = pd.DataFrame(data = ht)
    data_de = data_dd.dropna()
    data_df = data_de[(data_de['L1.qbfr'] != 'X') & (data_de['L2.qbfr'] != 'X') & (data_de['L3.qbfr'] != 'X') & (data_de['L4.qbfr'] != 'X') & (data_de['L0.acc'] > 1)]
    data[tool] = data_df

ValueError: All arrays must be of the same length

In [ ]:
sumdata = {
    'tool': [],
    'ltlcount': [],
    'autcount': [],
    'L0.acc': [],
}
for l in range(1, 7):
    for s in ['acc', 'time', 'ratio', 'qbft', 'qbfu']:
        sumdata['L' + str(l) + '.' + s] = []

for tool in tools_order:
    data_df = data[tool]
    
    sumdata['tool'].append(tool)
    sd = data_df.sum()
    sumdata['ltlcount'].append(ltlf_translated[tool])
    sumdata['autcount'].append(len(data_df))
    for i in range(0, 7):
        sumdata['L' + str(i) + '.acc'].append(int(sd['L' + str(i) + '.acc']))
        if i > 0:
            t = sd['L' + str(i) + '.time']
            #if i > 1 and i < 4:
            #    t -= sd['L' + str(i - 1) + '.time']
            sumdata['L' + str(i) + '.time'].append(round(t, 1))
            sumdata['L' + str(i) + '.ratio'].append(
                round(100 - 100 * int(sd['L' + str(i) + '.acc']) / int(sd['L0.acc']), 1)
            )
            
            for x in ['T', 'U']:
                sumdata['L' + str(i) + '.qbf' + x.lower()].append(len(data_df[data_df['L' + str(i) + '.qbfr'] == x]))

#for k in sumdata:
#    print(k, len(sumdata[k]))
r = pd.DataFrame(data = sumdata)
r

### Output